## Use netCDF4 package to read/write netCDF and HDF files with python

In [1]:
import netCDF4

### Write an "empty" netCDF file
- We usually use ".nc" for the file extension or ".nc4" to explicitlty refer to netCDF 4 data model
- However, the file extension is not what makes a file a netCDF file...

In [2]:
foo = netCDF4.Dataset("foo.nc", "w")
foo.close()

### An empty netCDF file is not empty... 
It contains default metadata and this is really what makes a file a netCDF file

In [3]:
foo = netCDF4.Dataset("foo.nc", "r")

In [4]:
print(foo)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: 



### Now write a new netCDF file called "xyz.nc"
- Create dimensions (here x, y and z)
- Create variables (oro)
- Fill variable oro using numpy array
- Close the netCDF file (to write data on disk)

In [5]:
f = netCDF4.Dataset("xyz.nc", "w")
f.createDimension('z',3)
f.createDimension('y',4)
f.createDimension('x',5)

<class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 5

In [6]:
print(f)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): z(3), y(4), x(5)
    variables(dimensions): 
    groups: 



### When creating a variable, you need to give:
- the variable dimensions
- its type to specify whether it is float or integer, etc.
- And you may add optional arguments; for instance to define compress (zlib=True)

In [7]:
lats = f.createVariable('lat',float, ('y',), zlib=True)
lons = f.createVariable('lon',float, ('x',), zlib=True)
oro = f.createVariable('orog', float, ('y','x'), zlib=True)

### Defined netCDF variables contain numpy arrays

In [8]:
lats[:] = [60,65, 70, 75]
lons[:] = [30, 60, 90, 120, 150]

In [9]:
import numpy as np

### To fill oro, we first define a vector and then reshape it to a 2D array

In [10]:
data_out = np.arange(4*5)

In [11]:
print(data_out)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [12]:
data_out.shape = (4,5)

In [13]:
print(data_out)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


In [14]:
oro[:] = data_out

### When you close your netCDF file, it writes it on disk

In [15]:
f.close()

### Open and read your netCDF file
- you don't have to read all the variables in memory
- it effectively reads and allocate memory when you access your data
- lats, lons and oro do not use a large amount of memory (even with large data dimensions) because they contain metadata only

In [16]:
f = netCDF4.Dataset("xyz.nc", "r")
lats = f.variables['lat']
lons = f.variables['lon']
oro = f.variables['orog']

In [17]:
print(oro)

<class 'netCDF4._netCDF4.Variable'>
float64 orog(y, x)
unlimited dimensions: 
current shape = (4, 5)
filling on, default _FillValue of 9.969209968386869e+36 used



### To read (and allocate memory), you need to access your variable:
- here we access the entire oro, lats and lon variables

In [18]:
print(oro[:])

[[  0.   1.   2.   3.   4.]
 [  5.   6.   7.   8.   9.]
 [ 10.  11.  12.  13.  14.]
 [ 15.  16.  17.  18.  19.]]


In [19]:
print(lats[:])
print(lons[:])

[ 60.  65.  70.  75.]
[  30.   60.   90.  120.  150.]


### But we could read a "slice" as we do with a numpy array
- Use "slices" to reduce the amount of memory used by your python code 

In [20]:
print(lats[1:2])

[ 65.]


## GeoTIFF are TIFF files with geo-location information
- you can use gdal to read a GeoTIFF file and get metadata 

In [21]:
from osgeo import gdal

In [22]:
datafile = gdal.Open('Southern_Norway_and_Sweden.2017229.terra.1km.tif')

In [23]:
print("Size of image:", datafile.RasterXSize, datafile.RasterYSize)

Size of image: 910 796


### Getting the projection used in the GeoTIFF file is very important for plotting and manipulating your data

In [24]:
print(datafile.GetProjectionRef())

GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]
